In [2]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import ranking_help_methods 
import make_question as question_maker
import vectorization
import sequence_handler
import corpus_handling_methods
import pandas as pd
import ranking
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
TFIDF_vectorizer = TfidfVectorizer()

In [1]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import make_question as question_maker
import corpus_handling_methods
import pandas as pd
import re

In [1]:
from gensim.models import Word2Vec
import numpy as np

gensim_model = Word2Vec.load('/home/iftekhar/myworkplace/AI-system/2VECs_models/word2vec/combined.model')

In [2]:
dataset = pd.read_csv("../../../Helpers/Title_link_merged_corpus.csv")
dataset = dataset.iloc[:,2:]
dataset = dataset.rename(columns={"Article": "Data"})
dataset.Data = dataset.Data.apply(lambda x: corpus_handling_methods.clean_text(x))
dataset.head()

,Data,Summary,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,MobiControl v14 Manual – MobiControl v14 Manual,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,Apple製品 – MobiControl v14 Manual ios overall,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,Android端末 – MobiControl v14 Manual plus overall,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,Windows PC – MobiControl v14 Manual wpc overall,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,Windows Embedded – MobiControl v14 Manual wm o...,4


In [3]:
split_corpus = corpus_handling_methods.corpus_split(dataset, 10)
split_corpus.head()

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,MobiControl 設定 順序 行い 端末 OS 対象 オンプレミス MobiContr...,0
2,のみ 必要 な 設定 事項 iOS 端末 macOS コンピュータ 必須,0
3,接続 プロファイル 作成 ADDS Active Directory Domain Serv...,0
4,ADFS Active Directory Federation Service 間 Azu...,0


In [4]:
len(split_corpus)

39400

In [6]:
perpage_dataset = corpus_handling_methods.corpus_per_page(split_corpus)
perpage_dataset.head() 

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,4


In [5]:
sample_size = 10
questions_samples = question_maker.question_dataframe_generator_1000(split_corpus, len(split_corpus))
questions_samples.head()

,Question,PageID
0,方法 参照 macOS 端末 構成 プロファイル メニュー 機能 拡張 どうですか,354
1,常時 接続 ない ので 長持ち ログ ファイル 管理 端末 どんな,76
2,証明 書 パスワード 選択肢 から 選択 接続 認証 パスワード なにで,283
3,INI ファイル バロメータ 値 XML ファイル 要素 ノード 入力 何ができますか,232
4,ヶ国 お客様 寄せ られ た 要望 凝縮 い 業務 どの,6


In [6]:
questions_samples.to_csv("questions_len_10.csv")

In [7]:
vec = vectorization.vector_fit(TFIDF_vectorizer, split_corpus)

In [9]:
sample_count = 0
sum_score = 0
container = []
extra_ranks = 0
for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    ranks = ranking.crude_ranks(perpage_sequence_match, query, vec)
    filtered_ranks = ranking.filtering_ranks(ranks, perpage_sequence_match, query, vec)
    
    if filtered_ranks[0][1] < 0.75:
        filtered_ranks = ranking.word2vec_ranks(perpage_sequence_match, query, gensim_model)
        print("Pretrained Model", col['PageID'], "<=>", filtered_ranks)
    else: 
        print(col['PageID'], "<=>", filtered_ranks)
        
    page_answers = []
    prediction_scores = []
    for ids, score in filtered_ranks:
        page_answers.append(ids)
        prediction_scores.append(score)
    MRR_score = ranking.mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score += MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
    sample_count += 1
    
result = pd.DataFrame(container, columns=['score', 'actual_answer',
'page_answers', 'prediction_scores', 'query'])
result.to_csv('seq_matcher_TFIDF_Vectorizer_performance.csv')
score = sum_score/sample_count
score



単独 使う ない 構文 sleep length length ことは何ですか 95
説明 端末 ロック 画面 消える 説明 ランチャー 使用しますか 243
Zebra ZTE アルファベット 順 ただし 古い 機種 何ができますか 6
線 矢印 クリック プルダウンメニュー 現れ 段階 から どんな 407
指定 Copyright Translated by Limits Co Ltd 使用しますか 409
アプリ 最初 起動 アプリ 紹介 使用 方法 使いたいです 191
消去 クッキー 機能 使う 可能 だ ブラウザ 何が 292
WEP WPA WPA 任意 パーソナル WEP WPA 作りたい 312
つ 詳しく 参照 端末 業務 アプリ 端末 どの 8
設定 項目 説明 インストール 方法 自動 セルフサービス 使うには 105
Microsoft Exchange サーバ アクセス 阻止 な アレンジ どうして 24
認証 失敗 アカウント 削除 なっ た 使う 開発を開始する場所 46


0.5416666666666666

In [14]:
filtered_ranks[0][1]

0.975616546550444

In [15]:
filtered_ranks

[[24, 0.975616546550444], [443, 0.975616546550444], [128, 0.4409370517422057]]